In [1]:
# system tools
import os

# for preprocessing
import cv2

# making arrays
import numpy as np

# visualizations
import matplotlib.pyplot as plt

# util functions
from imutils import jimshow as show
from imutils import jimshow_channel as show_channel
import classifier_utils as clf_util

# scikit-learn
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# cifar10 dataset
from tensorflow.keras.datasets import cifar10

2024-05-09 13:28:15.469690: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 13:28:15.476495: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 13:28:15.548739: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 13:28:16.784964: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train_list=[]
X_test_list=[]

for image in X_train: # preprocessing the training images
    gray_train = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
    normalized = gray_train/255 # normalize
    X_train_list.append(normalized) # add to list
X_preprocess_train_array = np.array(X_train_list) # turn list back into an array
X_preprocess_train = X_preprocess_train_array.reshape(-1, 1024) # reshape

for image in X_test: # preprocessing the testing images
    gray_test = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
    normalized = gray_test/255 # normalize
    X_test_list.append(normalized) # add to list
X_preprocess_test_array = np.array(X_test_list) # turn list back into an array
X_preprocess_test = X_preprocess_test_array.reshape(-1, 1024) # reshape

In [12]:
pipe = Pipeline([('classifier' , LogisticRegression())])

# Set tunable parameters for grid search
penalties = ['l1', 'l2'] # different regularization parameters
C = [1.0, 0.1, 0.01]     # different regularization 'strengths'
solvers = ['saga']  # different solvers - check all of the sklearn docs

# Create parameter grid (a Python dictionary)
parameters = dict(classifier__penalty = penalties,  # notice how we use the name 'classifier'
                  classifier__C = C,
                  classifier__solver = solvers)

# Choose which metrics on which we want to optimise
scores = ['precision', 'recall', 'f1']

In [8]:
# TEST PIPELINE

pipe = Pipeline([('classifier' , LogisticRegression())])

# Set tunable parameters for grid search
penalties = [None] # different regularization parameters
C = [1.0, 0.1, 0.01]     # different regularization 'strengths'
solvers = ['liblinear', 'saga']  # different solvers - check all of the sklearn docs

# Create parameter grid (a Python dictionary)
parameters = dict(classifier__penalty = penalties,  # notice how we use the name 'classifier'
                  classifier__C = C,
                  classifier__solver = solvers)

# Choose which metrics on which we want to optimise
scores = ['f1']

In [13]:
for score in scores:
    print(f"# Tuning hyper-parameters for {score}")
    print()
    
    # Initialise Gridsearch with predefined parameters
    clf = GridSearchCV(pipe, 
                       parameters, 
                       scoring= f"{score}_weighted",
                       cv=5) # use 10-fold cross-validation <------- changed to 5
    # Fit
    print("Begins fitting")
    clf.fit(X_preprocess_train, y_train)
    print("Done with fitting")
    
    # Print best results on training data
    print("Best parameters set found on training data:")
    # add new lines to separate rows
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on training data:")
    print()
    # get all means
    means = clf.cv_results_['mean_test_score']
    # get all standard deviations
    stds = clf.cv_results_['std_test_score']
    # get parameter combinations
    params = clf.cv_results_['params']

    # print means, standard deviations , and parameters for all runs
    i = 0
    for mean, stdev, param in zip(means, stds, params):
        # 2*standard deviation covers 95% of the spread - check out the 68–95–99.7 rule
        print(f"Run {i}: {round(mean,3)} (SD=±{round(stdev*2, 3)}), using {param}")
        i += 1
    print()
    
    # Print details classification report
    print("Detailed classification report:")
    print()
    print("The model is trained on the full training set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_preprocess_test)
    cm = metrics.classification_report(y_test, y_pred, target_names=classes)
    print(cm)
    print()

# Tuning hyper-parameters for precision

Begins fitting


/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataC

Done with fitting
Best parameters set found on training data:

{'classifier__C': 0.01, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}

Grid scores on training data:

Run 0: 0.291 (SD=±0.014), using {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Run 1: 0.285 (SD=±0.013), using {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Run 2: 0.302 (SD=±0.012), using {'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Run 3: 0.294 (SD=±0.015), using {'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Run 4: 0.263 (SD=±0.008), using {'classifier__C': 0.01, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}
Run 5: 0.305 (SD=±0.014), using {'classifier__C': 0.01, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}

Detailed classification report:

The model is trained on the full training set.
The scores are computed on the full test set.



TypeError: 'str' object is not callable

In [10]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [11]:
y_pred = classifier.predict(X_preprocess_test)

NameError: name 'classifier' is not defined

In [ ]:
cm = metrics.classification_report(y_test, y_pred, target_names=classes)
print(cm)

# saving classification report as a .txt file
text_file = open(r'output/LR_classification_report_gridsearch.txt', 'w')
text_file.write(cm)
text_file.close()
